In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from category_encoders import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

정리에 앞서, 한글번역을 할 경우에 혼동이 올 수 있기 때문에 영어 자체로 이해하는게 좋다고 생각합니다.

이번에는 scaling, normalize, onehotencoding, regularization 에 대해서 정리해보도록 하겠습니다.

위의 4가지를 상황에 맞게, 적재적소에 사용하는 것이 매우 중요하겠습니다.

그러기 위해선 원본 데이터의 설명을 잘, 꼼꼼히 읽어보고 EDA,Feature engineering 을 하는 것이 좋겠습니다.

또한 독립변수는 말 그대로 독립된 변수여야 하는데, 독립된 변수가 아닌 경우가 있습니다.

보통 컬럼명이 비슷한 경우 예) Condition1, Condition2 이런 것들은 만약 feature로 꼭! 반드시! 사용하고 싶다면

정말 두 변수가 독립인지를 확인해야 할 필요성이 있습니다. 

만약 두 변수가 서로 영향을 주고 받는다면 다중공산성의 문제가 발생할 수 있기 때문입니다.

미연에 방지하고자 저런 이름이 비슷한 컬럼은 보통 상관계수가 높은 1개만 사용하는 편 입니다.

Scalerd의 종류

1. StandardScaler -> 데이터의 값이 너무 들쭉날쭉하거나, 
                     값들이 차이가 너무 크거나 작을 때, 
                     변수들의 단위차이로 스케일이 크게 달라지는 경우 정상적인 학습이 이루어 질 수 없기 때문에
                     표준화를 해주는 친구입니다. (x-m/시그마)
                     특히 이 StandardScaler는 더 중요한 것이, 
                     나중에 정리할 서포트 벡터 머신이나 선형회귀,로지스틱 회귀 등 몇몇은
                     정규분포(가우시안 분포)를 띄고있다는 가정하에 구현됐기 때문에 
                     scaling을 해주는 것이 예측 성능을 향상 시킬 수 있을 것 입니다.

2. MinMaxScaler   -> 모든 값이 0과 1사이에 있도록 scaling 하는 친구입니다.

3. RobustSclaer   -> 평균,분산 대신 사분위 값과 중간값을 사용합니다.
                     X - Q2 / Q3 - Q1  (Q2 : 중간값, Q3 : 3분위값, Q1 : 1분위값)   

In [2]:
df = pd.read_csv("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/ames_home/ames_home_subset.csv")
df

,1stFlrSF,BedroomAbvGr,BldgType,BsmtFullBath,BsmtHalfBath,CentralAir,Condition1,Condition2,Electrical,ExterCond,...,RoofStyle,SalePrice,SaleCondition,SaleType,Street,TotRmsAbvGrd,Utilities,YearBuilt,YearRemod/Add,YrSold
0,1656,3,1Fam,1.0,0.0,Y,Norm,Norm,SBrkr,TA,...,Hip,215000,Normal,WD,Pave,7,AllPub,1960,1960,2010
1,896,2,1Fam,0.0,0.0,Y,Feedr,Norm,SBrkr,TA,...,Gable,105000,Normal,WD,Pave,5,AllPub,1961,1961,2010
2,1329,3,1Fam,0.0,0.0,Y,Norm,Norm,SBrkr,TA,...,Hip,172000,Normal,WD,Pave,6,AllPub,1958,1958,2010
3,2110,3,1Fam,1.0,0.0,Y,Norm,Norm,SBrkr,TA,...,Hip,244000,Normal,WD,Pave,8,AllPub,1968,1968,2010
4,928,3,1Fam,0.0,0.0,Y,Norm,Norm,SBrkr,TA,...,Gable,189900,Normal,WD,Pave,6,AllPub,1997,1998,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,1003,3,1Fam,1.0,0.0,Y,Norm,Norm,SBrkr,TA,...,Gable,142500,Normal,WD,Pave,6,AllPub,1984,1984,2006
2900,902,2,1Fam,1.0,0.0,Y,Norm,Norm,SBrkr,TA,...,Gable,131000,Normal,WD,Pave,5,AllPub,1983,1983,2006
2901,970,3,1Fam,0.0,1.0,Y,Norm,Norm,SBrkr,TA,...,Gable,132000,Normal,WD,Pave,6,AllPub,1992,1992,2006
2902,1389,2,1Fam,1.0,0.0,Y,Norm,Norm,SBrkr,TA,...,Gable,170000,Normal,WD,Pave,6,AllPub,1974,1975,2006


칼럼 설명
1stFlrSF: 1층 면적 (square feet)

BedroomAbvGr: 지하실 제외 침실 갯수

BldgType: 주거 형태

       1Fam Single-family Detached  
       2FmCon   Two-family Conversion; originally built as one-family dwelling
       Duplx    Duplex
       TwnhsE   Townhouse End Unit
       TwnhsI   Townhouse Inside Unit

BsmtHalfBath: 지하실 half-bathroom 개수 (세면대와 변기만 있는 화장실)

BsmtFullBath: 지하실 full bathroom 개수 (세면대, 변기, 샤워, 욕조 전부 있는 욕실)

CentralAir: 중앙 냉방 장치 유무

       N    No
       Y    Yes

Condition1: 다양한 조건과의 근접성

       Artery   Adjacent to arterial street
       Feedr    Adjacent to feeder street   
       Norm Normal  
       RRNn Within 200' of North-South Railroad
       RRAn Adjacent to North-South Railroad
       PosN Near positive off-site feature--park, greenbelt, etc.
       PosA Adjacent to postive off-site feature
       RRNe Within 200' of East-West Railroad
       RRAe Adjacent to East-West Railroad

Condition2: 다양한 조건과의 근접성 (하나 이상 존재할 시)

       Artery   Adjacent to arterial street
       Feedr    Adjacent to feeder street   
       Norm Normal  
       RRNn Within 200' of North-South Railroad
       RRAn Adjacent to North-South Railroad
       PosN Near positive off-site feature--park, greenbelt, etc.
       PosA Adjacent to postive off-site feature
       RRNe Within 200' of East-West Railroad
       RRAe Adjacent to East-West Railroad

Electrical: 전기 시스템

       SBrkr    Standard Circuit Breakers & Romex
       FuseA    Fuse Box over 60 AMP and all Romex wiring (Average) 
       FuseF    60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP    60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix  Mixed

ExterCond: 외관 소재 현황 평가

       Ex   Excellent
       Gd   Good
       TA   Average/Typical
       Fa   Fair
       Po   Poor

ExterQual: 외관 소재 품질 평가

       Ex   Excellent
       Gd   Good
       TA   Average/Typical
       Fa   Fair
       Po   Poor

Exterior1st: 집 외부 소재

       AsbShng  Asbestos Shingles
       AsphShn  Asphalt Shingles
       BrkComm  Brick Common
       BrkFace  Brick Face
       CBlock   Cinder Block
       CemntBd  Cement Board
       HdBoard  Hard Board
       ImStucc  Imitation Stucco
       MetalSd  Metal Siding
       Other    Other
       Plywood  Plywood
       PreCast  PreCast 
       Stone    Stone
       Stucco   Stucco
       VinylSd  Vinyl Siding
       Wd Sdng  Wood Siding
       WdShing  Wood Shingles

Exterior2nd: 집 외부 소재 (하나 이상 존재할 시)

       AsbShng  Asbestos Shingles
       AsphShn  Asphalt Shingles
       BrkComm  Brick Common
       BrkFace  Brick Face
       CBlock   Cinder Block
       CemntBd  Cement Board
       HdBoard  Hard Board
       ImStucc  Imitation Stucco
       MetalSd  Metal Siding
       Other    Other
       Plywood  Plywood
       PreCast  PreCast
       Stone    Stone
       Stucco   Stucco
       VinylSd  Vinyl Siding
       Wd Sdng  Wood Siding
       WdShing  Wood Shingles

Foundation: 주택의 토대 종류

       BrkTil   Brick & Tile
       CBlock   Cinder Block
       PConc    Poured Concrete 
       Slab Slab
       Stone    Stone
       Wood Wood

FullBath: 지상층의 full-bathroom 갯수

Functional: 주택의 기능성 (공제가 보장되지 않는 한 일반적인 것으로 가정)

       Typ  Typical Functionality
       Min1 Minor Deductions 1
       Min2 Minor Deductions 2
       Mod  Moderate Deductions
       Maj1 Major Deductions 1
       Maj2 Major Deductions 2
       Sev  Severely Damaged
       Sal  Salvage only

GrLivArea: 지상 주거지역 면적 (sqaure feet)

HalfBath: 지상층 half-bathroom 갯수

Heating: 난방 종류

       Floor    Floor Furnace
       GasA Gas forced warm air furnace
       GasW Gas hot water or steam heat
       Grav Gravity furnace 
       OthW Hot water or steam heat other than gas
       Wall Wall furnace

HeatingQC: 난방 품질 및 상태

       Ex   Excellent
       Gd   Good
       TA   Average/Typical
       Fa   Fair
       Po   Poor

HouseStyle: 주거 형식

       1Story   One story
       1.5Fin   One and one-half story: 2nd level finished
       1.5Unf   One and one-half story: 2nd level unfinished
       2Story   Two story
       2.5Fin   Two and one-half story: 2nd level finished
       2.5Unf   Two and one-half story: 2nd level unfinished
       SFoyer   Split Foyer
       SLvl Split Level

KitchenAbvGr: 지상층 주방 갯수

KitchenQual: 주방 품질

       Ex   Excellent
       Gd   Good
       TA   Typical/Average
       Fa   Fair
       Po   Poor

LandContour: 소유지의 평탄도

       Lvl  Near Flat/Level 
       Bnk  Banked - Quick and significant rise from street grade to building
       HLS  Hillside - Significant slope from side to side
       Low  Depression

LandSlope: 소유지 경사도

       Gtl  Gentle slope
       Mod  Moderate Slope  
       Sev  Severe Slope

LotArea: 용지 크기 (square feet)

LotConfig: 용지 구성

       Inside   Inside lot
       Corner   Corner lot
       CulDSac  Cul-de-sac
       FR2  Frontage on 2 sides of property
       FR3  Frontage on 3 sides of property

LotShape: 소유지의 형태
       Reg  Regular 
       IR1  Slightly irregular
       IR2  Moderately Irregular
       IR3  Irregular

MSSubClass: 분양되는 주거지의 유형    

        20  1-STORY 1946 & NEWER ALL STYLES
        30  1-STORY 1945 & OLDER
        40  1-STORY W/FINISHED ATTIC ALL AGES
        45  1-1/2 STORY - UNFINISHED ALL AGES
        50  1-1/2 STORY FINISHED ALL AGES
        60  2-STORY 1946 & NEWER
        70  2-STORY 1945 & OLDER
        75  2-1/2 STORY ALL AGES
        80  SPLIT OR MULTI-LEVEL
        85  SPLIT FOYER
        90  DUPLEX - ALL STYLES AND AGES
       120  1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150  1-1/2 STORY PUD - ALL AGES
       160  2-STORY PUD - 1946 & NEWER
       180  PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190  2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: 주택이 속해있는 구역 구분

       A    Agriculture
       C    Commercial
       FV   Floating Village Residential
       I    Industrial
       RH   Residential High Density
       RL   Residential Low Density
       RP   Residential Low Density Park 
       RM   Residential Medium Density

MasVnrType: 석조 베니어 유형

       BrkCmn   Brick Common
       BrkFace  Brick Face
       CBlock   Cinder Block
       None None
       Stone    Stone

MoSold: 팔린 달  

Neighborhood: Ames시의 경계내 물리적 위치

       Blmngtn  Bloomington Heights
       Blueste  Bluestem
       BrDale   Briardale
       BrkSide  Brookside
       ClearCr  Clear Creek
       CollgCr  College Creek
       Crawfor  Crawford
       Edwards  Edwards
       Gilbert  Gilbert
       IDOTRR   Iowa DOT and Rail Road
       MeadowV  Meadow Village
       Mitchel  Mitchell
       Names    North Ames
       NoRidge  Northridge
       NPkVill  Northpark Villa
       NridgHt  Northridge Heights
       NWAmes   Northwest Ames
       OldTown  Old Town
       SWISU    South & West of Iowa State University
       Sawyer   Sawyer
       SawyerW  Sawyer West
       Somerst  Somerset
       StoneBr  Stone Brook
       Timber   Timberland
       Veenker  Veenker

OverallCond: 주택의 전반적인 상태 평가

       10   Very Excellent
       9    Excellent
       8    Very Good
       7    Good
       6    Above Average   
       5    Average
       4    Below Average   
       3    Fair
       2    Poor
       1    Very Poor

OverallQual: 주택의 전체 자재와 마감재에 대한 평가

       10   Very Excellent
       9    Excellent
       8    Very Good
       7    Good
       6    Above Average
       5    Average
       4    Below Average
       3    Fair
       2    Poor
       1    Very Poor

PavedDrive: 포장 진입로

       Y    Paved 
       P    Partial Pavement
       N    Dirt/Gravel

RoofMatl: 지붕 소재

       ClyTile  Clay or Tile
       CompShg  Standard (Composite) Shingle
       Membran  Membrane
       Metal    Metal
       Roll Roll
       Tar&Grv  Gravel & Tar
       WdShake  Wood Shakes
       WdShngl  Wood Shingles

RoofStyle: 지붕 형태

       Flat Flat
       Gable    Gable
       Gambrel  Gabrel (Barn)
       Hip  Hip
       Mansard  Mansard
       Shed Shed

SalePrice: 주택의 판매 가격

SaleCondition: 판매 조건

       Normal   Normal Sale
       Abnorml  Abnormal Sale -  trade, foreclosure, short sale
       AdjLand  Adjoining Land Purchase
       Alloca   Allocation - two linked properties with separate deeds, typically condo with a garage unit  
       Family   Sale between family members
       Partial  Home was not completed when last assessed (associated with New Homes)

SaleType: 판매 유형

       WD   Warranty Deed - Conventional
       CWD  Warranty Deed - Cash
       VWD  Warranty Deed - VA Loan
       New  Home just constructed and sold
       COD  Court Officer Deed/Estate
       Con  Contract 15% Down payment regular terms
       ConLw    Contract Low Down payment and low interest
       ConLI    Contract Low Interest
       ConLD    Contract Low Down
       Oth  Other

Street: 소유지에 대한 도로 접근 유형

       Grvl Gravel  
       Pave Paved

TotRmsAbvGrd: 지상층 방 갯수 (화장실/욕실 제외)

Utilities: 수도세, 전기세, 가스세 포함 여부

       AllPub   All public Utilities (E,G,W,& S)    
       NoSewr   Electricity, Gas, and Water (Septic Tank)
       NoSeWa   Electricity and Gas Only
       ELO  Electricity only    

YearBuilt: 건축 연도

YearRemod/Add: 리모델링 / 추가 공사 연도 (아무런 리모델링이나 추가 시공이 없는 경우 건축연도와 동일)

YrSold: 판매 연도 (YYYY)

##########################################################################################################

MSSubClass 의 경우 수치가 어떤 높고 낮음을 의미하는 것이 아니라,

각 숫자에 해당하는 범주치가 있음을 확인할 수 있습니다.

저 경우에 컬럼의 설명을 읽지 않았다면 scaling을 해버리는 실수를 했을 것 입니다.

그래서 MSSubClass의 경우 타입을 Object타입으로 변경 후 OneHotEncoding을 진행해야 합니다.

Target 값을 SalePrice로 잡고 이 target값과 상관계수가 높은 컬럼들을 추출하여 ML 진행해보겠습니다.

In [3]:
# EDA 아까 MSSubClass의 경우 해결하기

df.MSSubClass = df.MSSubClass.astype(object)
df.OverallQual = df.OverallQual.astype(object)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2904 entries, 0 to 2903
Data columns (total 47 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       2904 non-null   int64  
 1   BedroomAbvGr   2904 non-null   int64  
 2   BldgType       2904 non-null   object 
 3   BsmtFullBath   2904 non-null   float64
 4   BsmtHalfBath   2904 non-null   float64
 5   CentralAir     2904 non-null   object 
 6   Condition1     2904 non-null   object 
 7   Condition2     2904 non-null   object 
 8   Electrical     2904 non-null   object 
 9   ExterCond      2904 non-null   object 
 10  ExterQual      2904 non-null   object 
 11  Exterior1st    2904 non-null   object 
 12  Exterior2nd    2904 non-null   object 
 13  Foundation     2904 non-null   object 
 14  FullBath       2904 non-null   int64  
 15  Functional     2904 non-null   object 
 16  GrLivArea      2904 non-null   int64  
 17  HalfBath       2904 non-null   int64  
 18  Heating 

In [5]:
# 상관계수 보기.
df.corr()['SalePrice'].sort_values(ascending = False)

SalePrice        1.000000
GrLivArea        0.708095
1stFlrSF         0.625522
YearBuilt        0.558414
FullBath         0.545927
YearRemod/Add    0.532115
TotRmsAbvGrd     0.496314
HalfBath         0.282205
BsmtFullBath     0.276769
LotArea          0.265802
BedroomAbvGr     0.144901
MoSold           0.030698
YrSold          -0.027753
BsmtHalfBath    -0.034730
OverallCond     -0.099841
KitchenAbvGr    -0.120238
Name: SalePrice, dtype: float64

수치형 컬럼에는 StandardScaler를, 범주형 컬럼에는 OneHotEncoding을 시켜주도록 하겠습니다.

In [6]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state = 42)
train_set, val_set = train_test_split(train_set, test_size=0.2, random_state = 42)

데이터 셋의 size가 작을 경우 예측의 신뢰도가 낮아지기 때문에

그런 경우라면 지금 사용하는 3 Way hold out보다 cross validation을 사용하여 train set의 size를 높여 어느정도 보완하는 것이 좋겠습니다.

In [7]:
numeric = ['1stFlrSF','GrLivArea','YearBuilt','FullBath','YearRemod/Add']
catagory = ['OverallQual', 'MSSubClass']
features = ['1stFlrSF','GrLivArea','OverallQual','YearBuilt','FullBath','YearRemod/Add','MSSubClass'] # 독립변수는 2차원 배열로
target = 'SalePrice' # 종속 변수는 1차원 배열로

def x_y_split(df) :
    X = df[features]
    y = df[target]
    return X, y

X_train, y_train = x_y_split(train_set)
X_val, y_val = x_y_split(val_set)
X_test, y_test = x_y_split(test_set)

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape


((1858, 7), (1858,), (465, 7), (465,), (581, 7), (581,))

In [8]:
# 수치형 feature들은 sacling을 진행하고, 범주형 컬럼들은 OneHotEncoding 하겠습니다.

scaler = StandardScaler()
X_train[numeric] = scaler.fit_transform(X_train[numeric]) # train set에만 fit.transform 진행하는거 잊지말아주세요 !
X_val[numeric] = scaler.transform(X_val[numeric])
X_test[numeric] = scaler.transform(X_test[numeric])

In [10]:
# 범주형 컬럼들을 OneHotEncoding하겠습니다. 수치형 컬럼들은 적용되지 않기 때문에 나누지 않고 진행하면 되겠습니다.

ohe = OneHotEncoder(use_cat_names= True)
X_train_scaled = ohe.fit_transform(X_train)
X_val_scaled = ohe.transform(X_val)
X_test_scaled = ohe.transform(X_test)

In [28]:
# 평가 모델 함수 만들기.


def eval_models(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred).round(3)
    print(f"mse = {mean_squared_error(y_true, y_pred).round(3)}")
    print(f"rmse = {np.sqrt(mse).round(3)}")
    print(f"tmae = {mean_absolute_error(y_true, y_pred).round(3)}")
    print(f"r2 = {r2_score(y_true, y_pred)}")

In [30]:
linreg = LinearRegression() # 여기서 파라미터 normalize = True를 해서 노멀라이즈 하는 경우, 각각의 변량에서 평균을 빼고 l2-norm으로 나누는 노멀라이즈를 하게 됨.
                            # 스케일링을 원할 경우 StandardScaler를 이용해야 합니다. 매우매우매우 중요
linreg.fit(X_train_scaled, y_train)
y_pred = linreg.predict(X_test_scaled);

In [31]:
eval_models(y_test,y_pred)

mse = 1087478150.014
rmse = 32976.934
tmae = 20885.316
r2 = 0.8372795379168214


r2 점수를 높이기 위해 검증세트를 활용하여 하이퍼 파라미터를 수정해야 할 것 같습니다.

Regularization은 Ridge,Lasso 등 에서 파라미터에 람다값을 넣어 가중치들을 조절하는 역할입니다.

ridgereg = Ridge(alpha = k, normalize = True)

여기서 alpha 가 람다에 해당합니다.